# Getting Heads 😶
## By Cody Kingham & Christiaan Erwich

## Problem Description
The ETCBC's BHSA core data does not contain the standard syntax tree format. This also means that syntactic and functional relationships between individual words are not mapped in a transparent or easily accessible way. In some cases, fine-grained relationships are ignored altogether. For example, for a given noun phrase (NP), there is no explicit way of obtaining its head noun (i.e. the noun itself without any modifying elements). This causes numerous problems for research in the realm of semantics. For instance, it is currently very difficult to calculate the complete person, gender, and number (PGN) of a given subject phrase. That is because PGN is stored at the word level only. But this is a very inadequate representation. Phrases in the ETCBC often contain coordinate relationships within the phrase. So even if one selects the first "noun" in the phrase and checks for its PGN value, they may overlook the presence of another noun which makes the phrase plural. Ideally, the phrase itself would have a PGN feature. But before this kind of data is created, it is necessary to separate the head words of a phrase from their modifying elements such as adjectives, determiners, or nouns in construct (genitive) relations.

A head word can be defined as the word for which a phrase type is named after. A phrase type can be NP for noun phrase or VP for verb phrase. In this notebook, we experiment with and build the functions stored in `heads.py` in order to export a set of Text-Fabric edge features. The edge features represent a mapping from a phrase node to its head element. 

This goal requires us to think carefully about the way inter-word, semantic relations are reflected in the ETCBC's data. The ETCBC *does* contain some rudimentary semantic embeddings through the so-called [subphrase](https://etcbc.github.io/bhsa/features/hebrew/c/otype). These can be utilized to isolate head words from secondary elements. A subphrase should *not* be thought of as a smaller, embedded phrase, like the ETCBC's phrase-atom (though it sometimes must indadequately fill that role). Rather, the subphrase is a way to encode relationships between words below the level of a phrase(atom), hence "sub." A subphrase can be a single word, or it can be a collection of words. A word can be in multiple subphrases, but can not be in more than 3 (due to the limitations of the data creation program, [parsephrases](http://www.etcbc.nl/datacreation/#ps3.p)).

## Method
The types of phrases represented in the ETCBC include `NP` (noun phrase), `VP` (verb phrase), `PrNP` (proper noun phrase), `PP` (prepositional phrase), `AdvP` (adverbial phrase), and [eight others](https://etcbc.github.io/bhsa/features/hebrew/c/typ). For some of these types, isolating the head word is a simple affair. By coordinating a word's phrase-dependent part of speech with its enclosing phrase's type, one can identify the head word. For a `VP`, that would mean simply finding the word within the phrase that has a `pdp` (phrase dependent part of speech) value of `verb`. Or for a prepositional phrase, find the word with a `pdp` of `prep`.

The `NP` and `PrNP`, on the other hand, present special challenges. These phrases often contain multiple words with a modifying relation to the head noun. An example of this is the construct relation (e.g. "Son of Jacob"). The problem becomes particularly thorny when relations like the construct are chained together so that one is faced with the choice between multiple potential head nouns.

To navigate the problem, we must use the feature [rela (relationship)](https://etcbc.github.io/bhsa/features/hebrew/c/rela) stored on `subphrase`s in addition to the `pdp` and phrase `type` features. In order to isolate the head word of a `NP`, we look for a word within the phrase that has a `pdp` value of `subs` (i.e. noun). We then obtain a list of all the `subphrase`s which contain that word using the [L.u Text-Fabric method](https://github.com/Dans-labs/text-fabric/wiki/Api#locality). We then use the list of subphrase node numbers to create a list of all subphrase relations containing the word. If the list contains *any* dependent relations, then the word is automatically excluded from being a head word and we can move on to the next candidate. One final check is required for candidate words at the level of the `phrase`: the same procedure described above for `subphrase`s must be performed for `phrase_atom` relations. This means excluding words within a `phrase_atom` with a dependent relation to another `phrase_atom` within the `phrase`. If the head of a *`phrase_atom`* is being calculated, this step is not necessary.

There are only two possible `subphrase` or `phrase_atom` relations for a valid head word: `NA` or `par`/`Para`. `NA` means that no relation is reflected. The word is independent. The `par` (`subphrase`) and `Para` (`phrase_atom`) stands for parallel relations, i.e. coordinates. These words require one further test, that is, it must be verified that their mother (using the [edge feature](https://github.com/Dans-labs/text-fabric/wiki/Api#edge-features) "[mother](https://etcbc.github.io/bhsa/features/hebrew/c/mother.html)") is itself a head word. To do this step thus requires us to keep track of those words within the phrase which have been validated. We can do so with a simple list.

## Code Development

Below we experiment with the code and develop the functions that will extract the head nouns. This involves a good deal of manual inspections of the results before exporting the Text-Fabric features.

The code is written immediately below. Associated questions that arise while writing or evaluating the code are contained in the subsequent section.

In [1]:
import collections, sys, os

# Load Text-Fabric and B (visualizer)
# separate import of tf.fabric not needed with bhsa module
LOC = ('~/github', 'etcbc/bhsa', 'getting_heads')
sys.path.append(os.path.expanduser(f'{LOC[0]}/{LOC[1]}/programs'))
from bhsa import Bhsa 
B = Bhsa(*LOC)
B.api.makeAvailableIn(globals())

B.load('''
      book chapter verse
      typ pdp rela mother 
      function lex sp ls
       ''')

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="{CORPUS} feature documentation">Feature docs</a> <a target="_blank" href="https://etcbc.github.io/bhsa/api.html" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://github.com/Dans-labs/text-fabric/wiki/api" title="text-fabric-api">Text-Fabric API</a>


This notebook online:
<a target="_blank" href="http://nbviewer.jupyter.org/github/etcbc/lingo/blob/master/heads/getting_heads.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/etcbc/lingo/blob/master/heads/getting_heads.ipynb">GitHub</a>


In [38]:
def give_heads(phrase):
    '''
    Extracts and returns the heads of a supplied
    phrase or phrase atom based on that phrase's type
    and the relations reflected within the phrase.
    
    --input--
    phrase(atom) node number
    
    --output--
    tuple of head word node(s) 
    '''
    
    # mapping from phrase type to good part of speech values for heads
    head_pdps = {'VP': {'verb'},                   # verb 
                 'NP': {'subs'},                   # noun 
                 'PrNP': {'nmpr', 'subs'},         # proper-noun 
                 'AdvP': {'advb', 'nmpr', 'subs'}, # adverbial 
                 'PP': {'prep'},                   # prepositional 
                 'CP': {'conj', 'prep'},           # conjunctive
                 'PPrP': {'prps'},                 # personal pronoun
                 'DPrP': {'prde'},                 # demonstrative pronoun
                 'IPrP': {'prin'},                 # interrogative pronoun
                 'InjP': {'intj'},                 # interjectional
                 'NegP': {'nega'},                 # negative
                 'InrP': {'inrg'},                 # interrogative
                 'AdjP': {'adjv'}                  # adjective
                } 
    
    # get phrase-head's part of speech value and list of candidate matches
    phrase_type = F.typ.v(phrase)
    head_candidates = [w for w in L.d(phrase, 'word')
                          if F.pdp.v(w) in head_pdps[phrase_type]]
        
    # VP with verbs require no further processing, return the head verb
    if phrase_type == 'VP':
        return tuple(head_candidates[0])
        
    # go head-hunting!
    heads = []
    
    for word in head_candidates:
        
        # gather the word's subphrase (+ phrase_atom if otype is phrase) relations
        word_phrases = list(L.u(word, 'subphrase'))
        word_phrases += list(L.u(word, 'phrase_atom')) if (F.otype.v(phrase) == 'phrase') else list()
        word_relas = set(F.rela.v(phr) for phr in word_phrases) or {'NA'}
        
        # check (sub)phrase relations for independency
        if word_relas - {'NA', 'par', 'Para'}: 
            continue
            
        # check parallel relations for independency
        elif word_relas & {'par', 'Para'} and mother_is_head(word_phrases, heads):
            heads.append(word)
            
        # save all others as heads, check for quantifiers first
        else:
            this_head = find_quantified(word) or word # check for quantifiers (e.g. כל); else take the word
            heads.append(this_head)
            
    return heads
            
def mother_is_head(word_phrases, previous_heads):
    
    '''
    Test and validate parallel relationships for independency.
    Must gather the mother for each relation and check whether 
    the mother contains a head word. 
    
    --input--
    * list of phrase nodes for a given word (includes subphrases)
    * list of previously approved heads
    
    --output--
    boolean
    '''
    
    # get word's enclosing phrases that are parallel
    parallel_phrases = [ph for ph in word_phrases if F.rela.v(ph) in {'par', 'Para'}]
    
    # get the mother for the parallel phrases
    parallel_mothers = [E.mother.f(ph)[0] for ph in parallel_phrases] 
    
    # get mothers' words, by mother
    parallel_mom_words = [set(L.d(mom, 'word')) for mom in parallel_mothers]
    
    # test for head in each mother
    test_mothers = [bool(phrs_words & set(previous_heads)) for phrs_words in parallel_mom_words] 
    
    return bool(test_mothers)
    
def find_quantified(word):
    
    '''    
    Check whether a head candidate is a quantifier (e.g. כל).
    If it is, walk down the construct chain until
    the quantified noun is found.
    
    --input--
    word node
    
    --output--
    new word node or None
    '''
    
    custom_quants = {'KL/', 'M<V/'} # quantifiers, others?
    
    if F.lex.v(word) not in custom_quants and F.ls.v(word) != 'card':
        return None
        
    # climb down the construct chain
    quantified = word
    while E.mother.t(quantified) and F.rela.v(E.mother.t(quantified)[0]) == 'rec':
        quantified = E.mother.t(quantified)[0]
        
    return quantified

In [26]:
quants = list(l for l in F.otype.s('lex') if F.ls.v(l) == 'card')

for w in L.d(652840, 'word'):
    print(T.text([w]), w)

שֶׁ֣בַע  2217
שָׁנִ֔ים  2218
וּ 2219
שְׁמֹנֶ֥ה  2220
מֵאֹ֖ות  2221
שָׁנָ֑ה  2222


In [23]:
from heads import get_quantified, get_heads

In [18]:
good_sp = {'subs', 'nmpr', 'adjv'}
good_pdp = {'subs', 'nmpr'}

get_quantified(2217, good_pdp, good_sp, diagnose=True)

no rectum found at word 2217


In [20]:
#B.show([(w,) for w in L.d(quants[3], 'word')[:10]])

In [52]:
L.u(2217, 'subphrase')

(1300936, 1300938)

In [53]:
E.mother.t(1300936)

(1300937,)

In [41]:
T.text(L.d(1300936, 'word'))

'שֶׁ֣בַע '

## Tests

In this section, important questions are asked whose answers are needed to ensure the code is written correctly. The BHSA data is queried to answer them. These are questions like, "Do we need to check for relational independency for only noun phrases?" (no); and "does every phrase type have a word with a corresponding pdp?" (no).

### Make definitions available for tests:

In [66]:
# mapping from phrase type to its head part of speech
type_to_pdp = {'VP': 'verb', # verb 
               'NP': 'subs', # noun 
               'PrNP': 'nmpr', # proper-noun 
               'AdvP': 'advb', # adverbial 
               'PP': 'prep', # prepositional 
               'CP': 'conj', # conjunctive
               'PPrP': 'prps', # personal pronoun
               'DPrP': 'prde', # demonstrative pronoun
               'IPrP': 'prin', # interrogative pronoun
               'InjP': 'intj', # interjectional
               'NegP': 'nega', # negative
               'InrP': 'inrg', # interrogative
               'AdjP': 'adjv'} # adjective

### Test for non-NP phrases with valid pdp but invalid head

These tests demonstrate that subphrase relation checks are also needed for phrase types besides noun phrases. The only valid subphrase/phrase_atom relations for any potential head word is either `NA` or `par`/`Para`. While a few phrase types do not need additional relational checks, e.g. personal pronoun phrases, we can go ahead and consistently handle all phrases in the same way.

The only exception to the above rule is the `VP`, for which there are 14 cases of the `VP`'s head word (verb) that is also in a subphrase with a regens (`rec`) relation.

The operational question of these tests was:
> Are there cases in which a non-NP phrase(atom) contains a word with the corresponding pdp value, but which is probably not a head?

To answer the question, we first survey all cases where the phrase type's head candidate is in a subphrase with a relation that is not normally "independent." Based on the survey, we manually check the most pertinent phrase types and results. The tests reveal that, indeed, relation checks are needed for many phrase types.

In [57]:
def test_pdp_safe(phrase_object='phrase_atom'):
    
    '''
    Make a survey of phrase types and their matching pdp words,
    count what kinds of subphrase relations these words 
    occurr in. The survey can then be used to investigate 
    whether phrase types besides noun phrases require relationship
    checks for independency.
    '''
    
    pdp_relas_survey = collections.defaultdict(lambda: collections.Counter())
    headless = 0
    
    for phrase in F.otype.s(phrase_object):

        typ = F.typ.v(phrase) # phrase type

        # skip noun phrases
        if typ in {'NP', 'PrNP'}: 
            continue

        head_pdp = type_to_pdp[typ]

        maybe_heads = [w for w in L.d(phrase, 'word') 
                           if F.pdp.v(w) == head_pdp]
        
        # this check shows that many
        # phrases don't have a word 
        # with a corresponding pdp!
        if not maybe_heads:
            headless += 1

        # survey the candidate heads' relations
        for word in maybe_heads:

            head_name = typ + '|' + head_pdp
            subphrases = L.u(word, 'subphrase')
            sp_relas = set(F.rela.v(sp) for sp in subphrases)\
                        if subphrases else {'NA'} # <- handle cases without any subphrases (i.e. verbs)

            pdp_relas_survey[head_name].update(sp_relas)

    print(f'phrases without matching pdp: {headless}\n')
    print('subphrase relation survey: ')
    for name, rela_counts in pdp_relas_survey.items():

        print(name)

        for r, count in rela_counts.items():
            print('\t', r, '-', count)

In [58]:
# for phrase_atoms
test_pdp_safe()

phrases without matching pdp: 837

subphrase relation survey: 
PP|prep
	 NA - 64539
	 par - 3822
	 adj - 42
	 rec - 9
VP|verb
	 NA - 69011
	 rec - 14
	 par - 1
CP|conj
	 NA - 53848
AdvP|advb
	 NA - 5137
	 par - 102
	 mod - 49
	 adj - 1
AdjP|adjv
	 NA - 1854
	 par - 138
	 atr - 6
	 adj - 3
	 rec - 1
InjP|intj
	 NA - 1872
	 par - 11
DPrP|prde
	 NA - 790
NegP|nega
	 NA - 6743
PPrP|prps
	 NA - 4468
	 par - 9
IPrP|prin
	 NA - 797
	 par - 1
InrP|inrg
	 NA - 1291
	 par - 3


In [59]:
# and for phrases
test_pdp_safe(phrase_object='phrase')

phrases without matching pdp: 670

subphrase relation survey: 
PP|prep
	 NA - 62330
	 par - 3676
	 adj - 42
	 rec - 10
VP|verb
	 NA - 69011
	 rec - 14
	 par - 1
CP|conj
	 NA - 52537
AdvP|advb
	 NA - 5089
	 par - 101
	 mod - 46
	 adj - 1
AdjP|adjv
	 NA - 1803
	 par - 119
	 atr - 6
	 adj - 3
	 rec - 1
InjP|intj
	 NA - 1872
	 par - 11
DPrP|prde
	 NA - 791
NegP|nega
	 NA - 6743
PPrP|prps
	 NA - 4388
	 par - 9
IPrP|prin
	 NA - 797
	 par - 1
InrP|inrg
	 NA - 1291
	 par - 3


^ These surveys tell us that for several of these phrase types, e.g. `InjP`, we can automatically take the word with the pdp value that corresponds with its phrase type as the head.

There are also quite a few cases where the phrase type does not have a word with a matching pdp value: 837 for phrase atoms and 670 for phrases. In the subsequent section we will run tests to find out why this is the case.

Back to the question of this section: There are 14 examples of VP with verbs that have a `rec` (nomen regens) relation. Are these heads or not? We check now...

In [60]:
def find_and_show(search_pattern):
    results = sorted(B.search(search_pattern))
    print(len(results), 'results')
    B.show(results)

In [12]:
# run notebook locally to see HTML-formatted results for the below searches


rec_verbs = '''

phrase_atom typ=VP
    subphrase rela=rec
        word pdp=verb
'''

#find_and_show(rec_verbs) # uncomment me!

In all 14 results, the verb serves as the true head word of the `VP`.

*Note: The verb will prove to be an exception, as all other words in a `rec` relation are not head words*

The `PP` also has some strange relations. We see what's going on with the same kind of inspection. First we look at the `rec` (regens) relations.

In [69]:
rec_preps = '''

phrase_atom typ=PP
    subphrase rela=rec
        word pdp=prep
'''

#find_and_show(rec_preps) #uncomment me!

The PP is different. In cases where the `phrase_atom` = `rec`, the preposition is *not* the head. Thus, the algorithm will need to check for these cases.

Now for the `adj` subphrase relation in `PP`:

In [14]:
adj_preps = '''

phrase_atom typ=PP
    subphrase rela=adj
        word pdp=prep
'''

#find_and_show(adj_preps) # uncomment me!

The results above show that the `adj` subphrase relation is also a non-head. These cases have to be excluded.

Now we move on to test the **adverb** relations reflected in the survey...

In [15]:
adv_adj = '''

phrase_atom typ=AdvP
    subphrase rela=adj
        word pdp=advb

'''

#find_and_show(adv_adj) # uncomment me!

The `adj` relationships in the adverbial phrase is also not a true head. Now for the `mod` (modifier) relation.

In [16]:
adv_mod = '''

phrase_atom typ=AdvP
    subphrase rela=mod
        word pdp=advb

'''

#find_and_show(adv_mod) # uncomment me!

In this case, it appears that `mod` is also an invalid relation for adverb phrases. And example is גם הלם ('also here') where גם is the adverb in `mod` relation, but the head is really הלם "here" (also an adverb). In several cases, the modifier modifies a verb. In these cases the "head," often a participle or infinitive, acts as the adverb, even though it is not explicitly marked as such.

Now we move on to the last examination, that of the `AdjP` (adjective phrase). There are three relations of interest:
> atr - 6 <br>
> adj - 3 <br>
> rec - 1 <br>

In [17]:
adj_atr = '''

phrase_atom typ=AdjP
    subphrase rela=atr
        word pdp=adjv

'''

#find_and_show(adj_atr) # uncomment me!

In [18]:
adj_adj = '''

phrase_atom typ=AdjP
    subphrase rela=adj
        word pdp=adjv

'''

#find_and_show(adj_adj) # uncomment me!

In [19]:
adj_rec = '''

phrase_atom typ=AdjP
    subphrase rela=rec
        word pdp=adjv

'''

#find_and_show(adj_rec) # uncomment me!

The results for the three searches above show indeed that the relations of `atr`, `adj`, and `rec` are not head words.

### Tests for phrase types without a word that has a valid pdp value

The initial survey above revealed that 837 phrase atoms and 670 phrases lack a word with a corresponding pdp value. Here we investigate to see why that is the case. Is there a way to compensate for this problem? Are these truly phrases that lack heads?

We run another survey and count the phrase types against the non-matching pdp values found within them. At this point, we must also exclude words that have dependent relations (as defined above, subphrase values of NA or parallel).

In [22]:
count_no_pdp = collections.defaultdict(lambda: collections.Counter())
record_no_pdp = collections.defaultdict(lambda: collections.defaultdict(list))

for phrase in F.otype.s('phrase_atom'):
    
    typ = F.typ.v(phrase)
    
    # see if there is not corresponding pdp value
    corres_pdp = type_to_pdp[typ]
    corresponding_pdps = [w for w in L.d(phrase, 'word') 
                             if F.pdp.v(w) == corres_pdp]
    
    if not corresponding_pdps:
        
        # put potential heads here
        maybe_heads = []
        
        # calculate subphrase relations
        for word in L.d(phrase, 'word'):
            
            # get subphrase relations
            word_subphrs = L.u(word, 'subphrase')
            sp_relas = set(F.rela.v(sp) for sp in word_subphrs) or {'NA'}
            
            # check subphrase relations for independence
            if sp_relas == {'NA'}:
                maybe_heads.append(word)
                
            # test parallel relation for independence
            elif sp_relas == {'NA', 'par'} or sp_relas == {'par'}:
                
                # check for good, head mothers
                good_mothers = set(sp for w in maybe_heads for sp in L.u(w, 'subphrase'))
                this_daughter = [sp for sp in word_subphrs if F.rela.v(sp) == 'par'][0]
                this_mother = E.mother.f(this_daughter)
                
                if this_mother in good_mothers:
                    maybe_heads.append(word)
                    
        # sanity check
        # maybe_heads should have SOMETHING
        if not maybe_heads:
            raise Exception(f'phrase {phrase} looks HEADLESS!')
        
        # count pdp types
        head_pdps = [F.pdp.v(w) for w in maybe_heads]
        count_no_pdp[typ].update(head_pdps)
        
        # save for examination
        for word in maybe_heads:
            record_no_pdp[typ][F.pdp.v(word)].append((phrase, word))
        
for name, counts in count_no_pdp.items():

    print(name)

    for pdp, count in counts.items():
        print('\t', pdp, '-', count)

AdvP
	 nmpr - 253
	 subs - 499
	 art - 190
	 conj - 13
PrNP
	 subs - 9
	 art - 3
CP
	 prep - 85
	 subs - 79
	 advb - 6
NP
	 adjv - 1
	 intj - 1


These results are a bit puzzling. The numbers here are words within the phrase atoms that have NO subphrase relations. That means, for example, words such as הַ "the" do not appear to have any subphrase relation to their modified nouns. That again illustrates the shortcoming of the ETCBC data in this respect. There should be a relation from the article to the determined noun.

From this point forward, I will begin working through all four phrase types and the cases reflected in the survey.

Beginning with the `AdvP` type and the article. Upon some initial inspection, I've found that in many of the `AdvP` with the article, there is also a substantive (`subs`) that was found by the search. Are there any cases where there is no `nmpr` or `subs` found alongside the article? We can use the dict `record_no_pdp` which has recorded all cases reflected in the survey. Below I look to see if all 190 cases of an article in these `AdvP` phrases also has a corresponding noun.

In [23]:
no_noun = []

for phrase in record_no_pdp['AdvP']['art']:
    
    pdps = set(F.pdp.v(w) for w in L.d(phrase[0], 'word'))
    
    if not {'nmpr', 'subs'} & pdps:
        no_noun.append((phrase,))
        
print(len(no_noun), 'without nouns found...')

0 without nouns found...


There it is. So all cases of these articles can be discarded. In these cases, the noun serves as the head of the adverbial phrase. An example of this is when the noun marks the location of the action (hence adverb). 

Next, we check the conjunctions found in the adverbial phrases. Are any of those heads?

In [113]:
#B.show(record_no_pdp['AdvP']['conj']) # uncomment me!

All conjunctions in these `AdvP` phrases function to mark coordinate elements (only ו in these results). They can also be discarded as not possible heads.

Now we investigate the `PrNP` results with `subs` and `art`...

In [27]:
#B.show(record_no_pdp['PrNP']['subs']) # uncomment me!

In [29]:
#B.show(record_no_pdp['PrNP']['art']) # uncomment me!

The `art` relations reflected in the second search are not heads, but are all related to a substantive. All of the results in `subs` are heads. Thus, the only acceptable pdp for `PrNP` besides a proper noun is `subs`.

Now we dig into `CP` results. 85 of them have no `pdp` of conjunction, but have a preposition instead. Let's see what's going on...

In [31]:
#B.show(record_no_pdp['CP']['prep'][:20]) # uncomment me!

These are very interesting results. These conjunction phrases are made up of constructions like ב+עבור and ב+טרם. Together these words function as a conjunction, but alone they are prepositions and particles. Is it even possible in this case to say that there is a "head"?

It could be said that these combinations of words mean more than the sum of their parts; they are good examples of constructions, i.e. combinations of words whose meaning cannot be inferred simply from their individual words. Constructions illustrate the vague boundary between syntax and lexicon (cf. e.g. Goldberg, 1995, *Constructions*).

While these words are indeed marked as conjunction phrases, it is better in this case to analyze them as prepositional phrases (which they also are...this is another shortcoming of our data, or perhaps a mistake??). Thus, the head is the preposition, not the prepositional object.

We should expect that the remaining `subs` and `advb` groups are in fact the objects of those prepositions (and hence excluded). Let's test that assumption by looking for a preposition behind these words...

In [39]:
no_prep = []

for (phrase, word) in record_no_pdp['CP']['subs'] + record_no_pdp['CP']['advb']:
    
    possible_prep = word - 1
    
    if F.sp.v(possible_prep) != 'prep':
        no_prep.append((phrase, word))
        
print(f'subs|advb with no preceding prepositions: {len(no_prep)}')

subs|advb with no preceding prepositions: 0


Here we see. We can confirm that none of the substantives or adverbs will be the head of a conjunction phrase. A preposition is the only other kind of head for the `CP` besides a conjunction itself.

Finally, we're left with a last noun phrase (`NP`) for which no matching noun was found. The search found instead both `adjv` (adjective) and a `intj` (interjection). Let's see it.

In [48]:
#B.show(record_no_pdp['NP']['adjv']) # uncomment me

This ^ one actually looks like a mistake in the BHSA data. The adjective ידיד "beloved" functions here as a substantive, and hence should have a phrase dependent part of speech of `subs` not `adjv`. This phrase will not receive a head relation, because the underlying data itself is flawed and should be fixed. Then a new run of the algorithm will assign the proper head.

In [52]:
#B.show(record_no_pdp['NP']['intj']) # uncomment me

In this case, the word אוי "woe" functions like a noun. This thus appears to be another mislabeled `pdp` value, since it should read `subs`. This, like the previous example, will not receive a head value due to the mistake.

### Retrieving Quantified Words

Words that are quantified with other words such as כל or cardinal numbers like שבעה will be missed by the algorithm. Instead, the number itself is identified as the head. This is undesirable, since quantifiers offer descriptive information about the noun itself. But how to extract the quantified word?

In a previous algorithm used for quantified extraction, we looked for a nomen regens relation on the quantifier and located the noun within the related subphrase. A second look shows that this approach is flawed.

In the example below, the quantifier שבעה "seven" has no nomen regens relation:

In [43]:
B.show([(2217,)]) # uncomment me!


## Result 0
(**word** `2217`)


Rather than reflecting a regen/rectum relation, the second word שנים "years," the quantified noun, has a subphrase relation of `adj` "adjunct":

In [51]:
print(T.text(L.d(652840, 'word')))
print()

for sp in L.u(2218, 'subphrase'): # subphrases belonging to "years"
    print(sp, '(subphrase)')
    print('\t', T.text(L.d(sp,'word')))
    print('\t', 'rela:',  F.rela.v(sp))
    print()

שֶׁ֣בַע שָׁנִ֔ים וּשְׁמֹנֶ֥ה מֵאֹ֖ות שָׁנָ֑ה 

1300937 (subphrase)
	 שָׁנִ֔ים 
	 rela: adj

1300938 (subphrase)
	 שֶׁ֣בַע שָׁנִ֔ים 
	 rela: NA



It is questionable why this relationship is marked as an "adjunct." Let's see what other kinds of subphrase relations are reflected by quantifieds.

In [83]:
quant_relas = collections.defaultdict(lambda: collections.Counter())
quant_ex = collections.defaultdict(lambda: collections.defaultdict(list))
quants = [word for lex in F.ls.s('card') for word in L.d(lex, 'word')]

for word in quants:
    
    subphrases = L.u(word, 'subphrase')
    sp_daughters = [E.mother.t(sp) for sp in subphrases if E.mother.t(sp)]
    sp_daughters += [E.mother.t(word)] if E.mother.t(word) else list()
    sp_relas = [F.rela.v(sp[0]) for sp in sp_daughters]
    quant_relas[F.lex.v(word)].update(sp_relas)
    
    for rela in sp_relas:
        quant_ex[F.lex.v(word)][rela].append((L.u(word, 'phrase')[0], word))
    
for name, counts in quant_relas.items():

    print(name)

    for pdp, count in counts.items():
        print('\t', pdp, '-', count)

>XD/
	 par - 62
	 adj - 42
	 rec - 69
	 Appo - 1
	 Spec - 6
	 atr - 8
	 mod - 4
CNJM/
	 rec - 345
	 adj - 267
	 par - 109
	 mod - 14
	 atr - 5
	 Spec - 6
	 dem - 3
	 Sfxs - 1
>RB</
	 adj - 285
	 par - 100
	 rec - 113
	 atr - 8
	 Spec - 1
	 mod - 1
CB</
	 par - 87
	 adj - 268
	 rec - 169
	 mod - 1
	 dem - 1
	 atr - 23
	 Spec - 4
CLC/
	 par - 148
	 adj - 331
	 rec - 175
	 atr - 4
	 dem - 1
	 Spec - 1
M>H/
	 rec - 30
	 adj - 302
	 par - 217
	 atr - 2
CMNH/
	 adj - 128
	 par - 28
	 rec - 8
	 mod - 1
TC</
	 adj - 46
	 par - 43
	 rec - 22
XMC/
	 adj - 282
	 par - 175
	 rec - 96
	 mod - 1
	 Attr - 1
	 atr - 4
	 Spec - 2
<FRH/
	 adj - 96
	 par - 16
	 Spec - 1
<FR=/
	 adj - 11
	 par - 6
	 rec - 30
CC/
	 adj - 176
	 par - 75
	 rec - 103
	 atr - 4
<FRJM/
	 adj - 209
	 par - 149
	 Spec - 1
<FR/
	 adj - 97
	 par - 12
	 mod - 2
	 atr - 6
	 rec - 2
	 Spec - 2
<FRH=/
	 adj - 52
	 rec - 49
	 par - 10
	 dem - 2
	 Spec - 1
>LP=/
	 adj - 143
	 rec - 33
	 par - 198
	 Spec - 1
	 atr - 2
<CTJ/
	 adj - 13
	 r

In [84]:
B.show(quant_ex['>XD/']['rec'])


## Result 0
(**phrase** `657239`, **word** `9871`)



## Result 1
(**phrase** `657492`, **word** `10266`)



## Result 2
(**phrase** `659313`, **word** `13183`)



## Result 3
(**phrase** `662500`, **word** `17903`)



## Result 4
(**phrase** `663977`, **word** `20340`)


...


## Result 64
(**phrase** `891404`, **word** `400335`)



## Result 65
(**phrase** `897622`, **word** `412887`)



## Result 66
(**phrase** `904472`, **word** `426148`)



## Result 67
(**phrase** `904536`, **word** `426266`)



## Result 68
(**phrase** `904540`, **word** `426272`)


In [79]:
give_heads(702392)

[89424]

In [85]:
for w in L.d(662500, 'word'):
    print(T.text([w]), w)

אֶת־ 17894
שְׁתֵּ֤י  17895
נָשָׁיו֙  17896
וְ 17897
אֶת־ 17898
שְׁתֵּ֣י  17899
שִׁפְחֹתָ֔יו  17900
וְ 17901
אֶת־ 17902
אַחַ֥ד  17903
עָשָׂ֖ר  17904
יְלָדָ֑יו  17905


In [86]:
find_quantified(17903)

1304784

In [87]:
T.text(L.d(1304784, 'word'))

'עָשָׂ֖ר '

In [127]:
get_preps = '''
phrase typ=PP
    word pdp=prep
    <: word pdp=prep
'''

get_preps = sorted(B.search(get_preps))

len(get_preps)

852

In [123]:
L.d(806914, 'subphrase')

()

In [128]:
B.show(get_preps[:10])


## Result 0
(**phrase** `651572`, **word** `117`, **word** `118`)



## Result 1
(**phrase** `651572`, **word** `118`, **word** `119`)



## Result 2
(**phrase** `652621`, **word** `1885`, **word** `1886`)



## Result 3
(**phrase** `652655`, **word** `1932`, **word** `1933`)



## Result 4
(**phrase** `653121`, **word** `2741`, **word** `2742`)



## Result 5
(**phrase** `653210`, **word** `2898`, **word** `2899`)



## Result 6
(**phrase** `653318`, **word** `3128`, **word** `3129`)



## Result 7
(**phrase** `653418`, **word** `3392`, **word** `3393`)



## Result 8
(**phrase** `653509`, **word** `3594`, **word** `3595`)



## Result 9
(**phrase** `653547`, **word** `3683`, **word** `3684`)


In [93]:
give_heads(get_preps[0][0])

[6978]

In [95]:
F.lex.v(6978)

'>T'

In [98]:
E.mother.t(6978)

()

In [101]:
L.u(6980, 'subphrase')

(1302560,)

In [102]:
F.rela.v(1302560)

'NA'

In [103]:
E.mother.t(1302560)

(1302561,)